In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import math
import optuna
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn import preprocessing
from NFLUtils import NFLUtils
nfl_utils = NFLUtils()

%matplotlib inline

# ANN
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

import logging
# Set up logging
logging.basicConfig(level=logging.INFO)

# XGBoost 
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import seaborn as sns # confusion matrix

# Set device to GPU if available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def set_all_seeds(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

### Load CSV
cp Combined.csv ~/drive/Notes/ML/Pytorch/footballData/n

In [3]:
df = pd.read_csv("./footballData/CombinedSlidingWindow4.csv", index_col=False, low_memory=False)

# Shuffle dataFrame (don't do this?)
# df = shuffle(df, random_state=101)
# df.head()
df.info()

test_performance_size = 200
test_performance_df = df[df.shape[0]-test_performance_size:]
df = df[:df.shape[0]-test_performance_size]
print(f'df after perf set removed: {df.shape}')
print(f'df perf set size {test_performance_df.shape}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5308 entries, 0 to 5307
Columns: 193 entries, Unnamed: 0 to kick_punt_umap_dim_2
dtypes: float64(176), int64(14), object(3)
memory usage: 7.8+ MB
df after perf set removed: (5108, 193)
df perf set size (200, 193)


### Remove items from performance set where you have no odds data

In [4]:
# test_performance_df = test_performance_df.loc[test_performance_df['D_start_odds']!= 0]
# test_performance_size = test_performance_df.shape[0]
# test_performance_df.tail()

# Columns to use
(TODO)

In [5]:
df[:5]

,Unnamed: 0,Season,Date,Home_Team,H_Q1,H_Q2,H_Q3,H_Q4,H_OT,H_Final,...,D_scoring_xpa,D_scoring_xpp,D_Final,D_Final_Allowed,D_start_odds,D_halftime_odds,D_datediff,D_pythagorean,kick_punt_umap_dim_1,kick_punt_umap_dim_2
0,232,1995,1995-10-08,CIN,3,3,3,7,0,25.011029,...,1.813,0.384,12.571,16.114,0.0,0.0,0.0,0.114,6.694265,8.759236
1,229,1995,1995-10-08,ARI,3,7,11,0,0,18.544118,...,-1.504,0.180,-3.344,2.721,0.0,0.0,0.0,-0.063,10.335660,12.424065
2,228,1995,1995-10-08,WAS,10,7,7,10,0,15.308824,...,-0.695,-0.409,-2.291,-12.037,0.0,0.0,0.0,0.179,3.603538,2.439008
3,234,1995,1995-10-08,PIT,0,7,6,3,0,21.694853,...,-0.192,-0.244,10.599,9.664,0.0,0.0,0.0,0.248,7.307873,9.188602
4,223,1995,1995-10-09,LAC,3,10,0,10,0,18.268382,...,-0.256,0.000,-4.276,-10.798,0.0,0.0,0.0,0.007,2.883894,3.124932


## 1. Separate continuous, categorical, and label column names

Pretty much everything is continuous. 

Note: the y_col is what you're trying to predict

## Feature engineering
New Columns
- **h_win**: Home team won


In [6]:
cat_cols = []

cont_cols = [col for col in nfl_utils.cont_cols if col not in nfl_utils.drop_cols]


y_col = ['H_Won']
y_col_perf = ['H_Won', 'H_start_odds', 'V_start_odds', 'H_halftime_odds', 'V_halftime_odds']


# create cont_df and y_df from the df
print(df.shape)
cont_df = df[cont_cols]
y_df = df[y_col]

# test performance set
perf_conts_df = test_performance_df[cont_cols]
perf_y_df = test_performance_df[y_col_perf]
perf_date_df = test_performance_df[['Date','Home_Team', 'Visitor_Team']]

# print(cont_df.dtypes)
print(cont_df.shape)
print(perf_y_df.shape)
print(perf_y_df.tail())
print(perf_conts_df.tail())


(5108, 193)
(5108, 48)
(200, 5)
      H_Won  H_start_odds  V_start_odds  H_halftime_odds  V_halftime_odds
5303    0.0      3.805714      1.285714         1.263333         3.894444
5304    1.0      2.695000      1.475000         1.345556         3.222222
5305    0.0      1.785000      2.082143         1.301111         3.472222
5306    1.0      1.506000      2.654000         1.056667         9.383333
5307    0.0      1.809000      2.038000         4.491667         1.198333
      D_First_Downs  D_Rush   D_Yds   D_TDs  D_Cmp   D_Att    D_TD  D_Sacked  \
5303         -0.372 -13.742  -0.238 -10.965  6.426   8.056  28.902     4.363   
5304          0.069 -44.346  -3.191  11.977  2.841  21.311  -4.210    -0.159   
5305          1.647  32.044   4.502 -57.237  2.949 -36.737  58.033    -1.440   
5306         -4.958   6.489   4.651 -22.117 -8.421 -71.139  48.970    -0.714   
5307          0.141 -48.750 -12.522  40.661 -7.487  36.501 -25.872   -10.884   

      D_Yards  D_Net_Pass_Yards  ...  D_tac

## Monotone constraints dict

In [7]:
monotone_constraints = {
    # Time between games - More rest is better
    'D_datediff': 1,
    
    # Offensive positive indicators
    'D_First_Downs': 1,
    'D_Rush': 1,
    'D_Yds': 1,
    'D_TDs': 1,
    'D_Cmp': 1,
    'D_Yd': 1,
    'D_TD': 1,
    'D_Net_Pass_Yards': 1,
    'D_Total_Yards': 1,
    
    # Negative indicators
    'D_INT': -1,
    'D_Sacked': -1,
    'D_Fumbles': -1,
    'D_Lost': -1,
    'D_Turnovers': -1,
    'D_Penalties': -1,
    
    # Passing detail - positive
    'D_passing_cmp': 1,
    'D_passing_td': 1,
    'D_passing_yds': 1,
    'D_passing_lng': 1,
    
    # Passing detail - negative
    'D_passing_int': -1,
    'D_passing_sk': -1,
    
    # Receiving/Rushing - all positive
    'D_receiving_td': 1,
    'D_receiving_yds': 1,
    'D_receiving_lng': 1,
    'D_rushing_td': 1,
    'D_rushing_yds': 1,
    'D_rushing_lng': 1,
    
    # Defense - generally positive when in your favor
    'D_def_interceptions_int': 1,
    'D_def_interceptions_td': 1,
    'D_def_interceptions_yds': 1,
    'D_fumbles_fr': 1,
    'D_fumbles_td': 1,
    'D_sk': 1,
    # 'D_tackles_qbhits': 1,
    # 'D_tackles_tfl': 1,
    
    # Special teams - positive indicators
    'D_kick_returns_td': 1,
    'D_kick_returns_yds': 1,
    'D_punt_returns_td': 1,
    'D_punt_returns_yds': 1,
    
    # Scoring - positive
    'D_scoring_fgm': 1,
    'D_scoring_xpm': 1
}

monotone_constraints_tuple = ()
no_monotone_constraints_tuple = ()
for col in cont_cols:
    monotone_constraints_tuple = monotone_constraints_tuple + (monotone_constraints[col],) if col in monotone_constraints else monotone_constraints_tuple + (0,)
    no_monotone_constraints_tuple = no_monotone_constraints_tuple + (0,)
print(monotone_constraints_tuple)
print(no_monotone_constraints_tuple)

(1, 1, 1, 1, 1, 0, 1, -1, 0, 1, 1, -1, -1, -1, 0, 0, 0, 0, 1, -1, 1, -1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


#### 1a. Normalize cont_df
StandardScaler is instead used by the pipeline

In [8]:
# min_max_scaler = preprocessing.MinMaxScaler()
# cont_scaled = min_max_scaler.fit_transform(cont_df.values)
# cont_df = pd.DataFrame(cont_scaled)
# cont_df.head()

# # test performance set
# perf_conts_df_scaled = min_max_scaler.fit_transform(perf_conts_df.values)
# perf_conts_df = pd.DataFrame(perf_conts_df_scaled)

### 3. Create an array of continuous values
Numpy array 'conts' containing stack of each continuous column

In [9]:
conts = np.stack([cont_df[col].values for col in list(cont_df.columns)], 1)
conts[:5]

y_col = np.stack([y_df[col].values for col in y_col], 1)

# test performance set
perf_conts = np.stack([perf_conts_df[col].values for col in list(perf_conts_df.columns)], 1)
perf_y_col = np.stack([perf_y_df[col].values for col in list(perf_y_df.columns)], 1)
perf_date_col = np.stack([perf_date_df[col].values for col in list(perf_date_df.columns)], 1)


conts_train = conts
y_train = y_col

### 4. Convert conts to tensor

In [10]:
print(conts.shape)
print(y_col.shape)

(5108, 48)
(5108, 1)


In [11]:
# Handled by model.fit()
# conts = torch.tensor(conts, dtype=torch.float32)
# y_col = torch.tensor(y_col, dtype=torch.float32)

In [12]:
class TabularModelUpdated(nn.Module, BaseEstimator, ClassifierMixin):
    def __init__(self, n_cont, out_sz, layer_shape, p=0.5, criterion=nn.MSELoss(),
                optimizer_class=torch.optim.Adam, lr=0.001, confidence_threshold=0.1):
        super().__init__()
        # Model architecture params
        self.layer_shape = layer_shape
        self.n_cont = n_cont
        self.out_sz = out_sz
        self.p = p
        self.lr = lr
        
        # Training params
        self.criterion = criterion
        self.confidence_threshold = confidence_threshold
        
        # BatchNorm layer for continuous data
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        # Variable that holds the list of layers
        layerlist = []
        n_in = n_cont # no embed again
        # Iterate through the passed in "layers" parameter (ie, [200,100]) to build a list of layers
        for i, width in enumerate(self.layer_shape):
            # First layer gets special treatment
            if i == 0:
                layerlist.extend([
                    nn.Linear(n_in, width),
                    nn.Mish(),  # Mish instead of ReLU
                    nn.BatchNorm1d(width),
                    nn.Dropout(p/2)  # Less dropout in earlier layers
                ])
            else:
                layerlist.extend([
                    nn.Linear(n_in, width),
                    nn.Mish(),
                    nn.BatchNorm1d(width),
                    nn.Dropout(p)
                ])
            n_in = width
        # layerlist.append(nn.Linear(layers[-1], out_sz))
        
        # Final layer
        layerlist.extend([
            nn.Linear(self.layer_shape[-1], out_sz),
            # nn.Sigmoid()  # Ensures output between 0 and 1
        ])
        # Convert the list of layers into an attribute
        self.layers = nn.Sequential(*layerlist)
        # Initialize the optimizer
        self.optimizer = optimizer_class(self.parameters(), lr=self.lr)

        
    def forward(self, x_cont):
        x_cont = self.bn_cont(x_cont)  # Normalize the incoming continuous data
        x = self.layers(x_cont)        # Set up model layers
        return x
    
    def fit(self, X, y):
        """
        For sklearn pipeline
        """
        # Convert X,y to torch.tensor if needed
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X)
        if not isinstance(y, torch.Tensor):
            y = torch.FloatTensor(y)
        
        # Initialize optimizer
        # optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        # Training loop
        self.train()
        self.optimizer.zero_grad()
        y_pred = self.forward(X)
        loss = self.criterion(y_pred, y)
        loss.backward()
        self.optimizer.step()
        return self
    
    def predict(self, X):
        probas = self.predict_proba(X)
        return (probas > 0.5).astype(int)
        # return probas
    
    def predict_proba(self, X):
        self.eval()
        with torch.no_grad():
            if not isinstance(X, torch.Tensor):
                X = torch.FloatTensor(X)
            return self(X).numpy()
    
    def score(self, X, y):
        """        
        12/5 - this isn't called at all if 'scoring' is defined
        """
        probas = self.predict_proba(X)
        # First apply confidence thresholding
        mask = (probas < 0.5 - self.confidence_threshold) | (probas > 0.5 + self.confidence_threshold)
        predictions = np.where(mask, (probas > 0.5).astype(np.int32), np.nan)

        # Use numpy mask for nan values
        valid_mask = ~np.isnan(predictions)
        valid_predictions = predictions[valid_mask]
        valid_targets = y[valid_mask]
        
        # Penalize if < 80% predicted
        if (1.0* len(valid_predictions) / len(X)) < 0.85:
            # print(f"mask {len(valid_predictions)}  pred {len(X)}")
            return 0.0

        # Apply f1 score
        score = f1_score(valid_predictions.flatten(), valid_targets)
        return score

In [17]:
def objective(trial):
    # --- Suggest hyperparameters ---

    criterion = trial.suggest_categorical('criterion', nfl_utils.map_losses(None).keys())
    first_layer_size = trial.suggest_categorical('first_layer_size', [64, 56, 48, 32, 16, 12])
    min_layers = math.floor(math.sqrt(first_layer_size))
    num_layers = trial.suggest_int('num_layers', 2, min_layers)
    confidence_threshold = trial.suggest_float('confidence_threshold', 0, 0.05)
    layer_shape = [first_layer_size]
    for i in range(1, num_layers):
        layer_shape.append(first_layer_size//(2*i))
    
    # Set random state to have consistent results (42 is arbitrary)
    set_all_seeds()
    
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=False)

    # Split once
    X_train_fold = []
    X_val = []
    y_train_fold = []
    y_val = []
    models = []
    for train_index, val_index in kf.split(conts_train):
        # print(f"train {train_index.shape} val {val_index.shape}")
        X_train_fold.append(torch.FloatTensor(conts_train[train_index]).to(device))
        X_val.append(torch.FloatTensor(conts_train[val_index]).to(device))

        y_train_fold.append(torch.FloatTensor(y_train[train_index]).to(device))
        y_val.append(torch.FloatTensor(y_train[val_index]).to(device))

        model = TabularModelUpdated(
            n_cont=conts.shape[1],
            out_sz=1,
            layer_shape=layer_shape,
            p=trial.suggest_float('dropout', 0.28, 0.38),     # Dropout
            criterion=nfl_utils.map_losses(criterion),
            optimizer_class=torch.optim.Adam,
            lr=trial.suggest_float('lr', 1e-3, 1e-2, log=True),   # Learning rate 
            confidence_threshold=confidence_threshold
        )
        pipeline = Pipeline([
            ('scaler', StandardScaler()), # Standardize the numerical features
            # ('regressor', LinearRegression()), # Apply a regression model
            ('model', model)
        ])
        models.append(pipeline)

    # Run once on each split, track average loss, stop if > max patience
    max_patience = 10
    current_patience = max_patience
    tracked_loss = 0.0
    n_epochs = 0
    while current_patience > 0 or n_epochs < 100:
        n_epochs = n_epochs + 1
        running_loss = []
        for i in range(0,n_splits):
            # ----- Train -----
            models[i].fit(X_train_fold[i], y_train_fold[i])

            # ----- Eval -----
            running_loss.append(models[i].score(X_val[i], y_val[i]))
            # y_pred = models[i].predict(X_val[i])
            # running_loss.append(f1_score(y_val[i], y_pred))
        running_loss = np.mean(running_loss)

        # ----- Current epoch loss > previous -----
        # print(f"{tracked_loss} {running_loss} {tracked_loss < running_loss}")
        if tracked_loss < running_loss:
            current_patience = max_patience
            tracked_loss = running_loss
        else:
            current_patience = current_patience - 1
    trial.suggest_int('n_epochs', n_epochs, n_epochs)
    trial.report(tracked_loss, n_epochs)
    return tracked_loss

def print_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value}")
    print(f"Best trial so far: {study.best_trial.number}, value: {study.best_trial.value}")

In [18]:
study = optuna.create_study(
    direction='maximize',   # max because using f1
    pruner=optuna.pruners.HyperbandPruner(
        min_resource=1,
        max_resource=1000
    )
)
# Uncomment to run
if True:
    study.optimize(objective, n_trials=2000, callbacks=[print_callback])
    # study.optimize(objective, n_trials=3)
    print("Best trial:")
    trial = study.best_trial
    print("Value: ", trial.value)
    print("Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")


[I 2025-03-02 20:11:20,641] A new study created in memory with name: no-name-571f9f4a-ad10-43d4-941e-6072fa33018e
[I 2025-03-02 20:11:33,575] Trial 0 finished with value: 0.44086642419114536 and parameters: {'criterion': 'L1Loss', 'first_layer_size': 48, 'num_layers': 2, 'confidence_threshold': 0.012538014874680786, 'dropout': 0.28124651361101594, 'lr': 0.004233958635390632, 'n_epochs': 144}. Best is trial 0 with value: 0.44086642419114536.


Trial 0 finished with value: 0.44086642419114536
Best trial so far: 0, value: 0.44086642419114536


[I 2025-03-02 20:11:49,850] Trial 1 finished with value: 0.14669276878833026 and parameters: {'criterion': 'SmoothL1Loss', 'first_layer_size': 64, 'num_layers': 4, 'confidence_threshold': 0.0402354807109669, 'dropout': 0.3748113362797768, 'lr': 0.0018748937719715632, 'n_epochs': 108}. Best is trial 0 with value: 0.44086642419114536.


Trial 1 finished with value: 0.14669276878833026
Best trial so far: 0, value: 0.44086642419114536


[I 2025-03-02 20:12:22,688] Trial 2 finished with value: 0.17864277704927434 and parameters: {'criterion': 'L1Loss', 'first_layer_size': 64, 'num_layers': 7, 'confidence_threshold': 0.04123791171255932, 'dropout': 0.3221050103964721, 'lr': 0.003924833837605191, 'n_epochs': 131}. Best is trial 0 with value: 0.44086642419114536.


...

Trial 269 finished with value: 0.5193929897459653
Best trial so far: 264, value: 0.5539537581136061


[I 2025-03-02 22:16:41,538] Trial 270 finished with value: 0.533199101737097 and parameters: {'criterion': 'SmoothL1Loss', 'first_layer_size': 48, 'num_layers': 5, 'confidence_threshold': 0.015070331118917434, 'dropout': 0.3239119099434909, 'lr': 0.009984890572737456, 'n_epochs': 100}. Best is trial 264 with value: 0.5539537581136061.


Trial 270 finished with value: 0.533199101737097
Best trial so far: 264, value: 0.5539537581136061


[W 2025-03-02 22:16:44,715] Trial 271 failed with parameters: {'criterion': 'MSELoss', 'first_layer_size': 48, 'num_layers': 5, 'confidence_threshold': 0.026504897641837676, 'dropout': 0.32938395941126625, 'lr': 0.009237860793868445} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/forbesjon2/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_23544/1515200931.py", line 60, in objective
    models[i].fit(X_train_fold[i], y_train_fold[i])
  File "/Users/forbesjon2/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_23544/2109550693.py", line 73, in fit
    y_pred = self.forward(X)
  File "/var/folders/k0/7b4qgkdx2vb

KeyboardInterrupt: 

In [ ]:
# best_params = study.best_trial.params
best_params = {'criterion': 'MSELoss', 'first_layer_size': 64, 'num_layers': 7, 'confidence_threshold': 1.8653941637231173e-05, 'dropout': 0.3181431308672629, 'lr': 0.009969598746996452, 'n_epochs': 231}
layer_shape = [best_params['first_layer_size']]
for i in range(1, best_params['num_layers']):
    layer_shape.append(best_params['first_layer_size']//(2*i))

# Set random state to have consistent results (42 is arbitrary)
set_all_seeds()
model = TabularModelUpdated(
    n_cont=conts.shape[1],
    out_sz=1,
    layer_shape=layer_shape,
    p=best_params['dropout'],     # Dropout
    criterion=nfl_utils.map_losses(best_params['criterion']),
    optimizer_class=torch.optim.Adam,
    lr= best_params['lr'],   # Learning rate 
    confidence_threshold=best_params['confidence_threshold']
)


pipeline = Pipeline([
    ('scaler', StandardScaler()), # Standardize the numerical features
    # ('regressor', LinearRegression()), # Apply a regression model
    ('model', model)
])

# Run once on each split, track average loss, stop if > max patience
for _ in range(0, (best_params['n_epochs'] - 10)):
    running_loss = 0.0
    # ----- Train -----
    pipeline.fit(conts_train, y_train)

    # ----- Eval -----
    # loss = pipeline.score(perf_conts, perf_y_col)
    #print(f"loss: {loss}")

# pipeline.fit(conts_train, y_train)
probas = pipeline.predict(perf_conts)
confidence_threshold = best_params['confidence_threshold']


Value:  0.37491718013436764

Params: 
    first_layer_size: 8
    num_layers: 2
    n_epochs: 39
    dropout: 0.47875406200808335
    lr: 0.009997751942238913
    


Value:  0.3759073484440955
Params: 
    first_layer_size: 8
    num_layers: 2
    n_epochs: 68
    dropout: 0.4497689844977892
    lr: 0.007977206154472633
    
    
    
12/6

Trial 206 finished with value: 0.547218605316966 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 32, 'num_layers': 5, 'confidence_threshold': 0.05966702820817666, 'n_epochs': 379, 'dropout': 0.36961850006275193, 'lr': 0.008649806179332952}. Best is trial 206 with value: 0.547218605316966.

[I 2024-12-06 12:49:28,047] Trial 579 finished with value: 0.5335308702482566 and parameters: {'criterion': 'SmoothL1Loss', 'first_layer_size': 16, 'num_layers': 3, 'confidence_threshold': 0.059979796814548306, 'n_epochs': 481, 'dropout': 0.2511747953677191, 'lr': 0.007942836869449217}. Best is trial 579 with value: 0.5335308702482566.


[I 2024-12-06 14:53:00,850] Trial 385 finished with value: 0.5547767877242975 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 56, 'num_layers': 6, 'confidence_threshold': 0.003263372268063613, 'n_epochs': 300, 'dropout': 0.3153661030384182, 'lr': 0.00593138298730814}. Best is trial 385 with value: 0.5547767877242975.

[I 2024-12-06 15:32:38,969] Trial 583 finished with value: 0.550872165273167 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 56, 'num_layers': 7, 'confidence_threshold': 0.010458110701511005, 'n_epochs': 336, 'dropout': 0.3188974735143638, 'lr': 0.006976522077116529}. Best is trial 385 with value: 0.5547767877242975.

[I 2024-12-06 15:55:26,133] Trial 669 finished with value: 0.5645423555160363 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 56, 'num_layers': 7, 'confidence_threshold': 0.0243907527056759, 'n_epochs': 370, 'dropout': 0.32998724447261185, 'lr': 0.0075018456671685696}. Best is trial 669 with value: 0.5645423555160363.

[I 2024-12-06 20:33:46,555] Trial 1737 finished with value: 0.5716002919237433 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 56, 'num_layers': 7, 'confidence_threshold': 0.015368961704784596, 'n_epochs': 328, 'dropout': 0.348927921024466, 'lr': 0.009575624984802092}. Best is trial 1737 with value: 0.5716002919237433.


[I 2024-12-06 21:17:06,545] Trial 1889 finished with value: 0.5739803740995499 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 56, 'num_layers': 7, 'confidence_threshold': 0.014029751567812504, 'n_epochs': 357, 'dropout': 0.34275064196127053, 'lr': 0.008692336113071646}. Best is trial 1889 with value: 0.5739803740995499.

[I 2024-12-10 09:39:54,796] Trial 1612 finished with value: 0.5748324966932515 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 64, 'num_layers': 7, 'confidence_threshold': 0.003855147984840053, 'dropout': 0.3182765851196762, 'lr': 0.008210651343970551, 'n_epochs': 219}. Best is trial 1612 with value: 0.5748324966932515.



Trial 1749 finished with value: 0.5772962775717783 and parameters: {'criterion': 'MSELoss', 'first_layer_size': 64, 'num_layers': 7, 'confidence_threshold': 1.8653941637231173e-05, 'dropout': 0.3181431308672629, 'lr': 0.009969598746996452, 'n_epochs': 231}.




In [ ]:
nfl_utils.backtest_model(pipeline, perf_conts, perf_y_col, perf_date_col, initial_capital=1000, position_size=0.05, confidence_threshold=best_params['confidence_threshold'], show_plot=True)


In [ ]:
# Remove outside of confidence threshold
mask = (probas < 0.5 - confidence_threshold) | (probas > 0.5 + confidence_threshold)
predictions = np.where(mask, probas, np.nan)

# Use numpy mask for nan values
valid_mask = ~np.isnan(predictions)
valid_predictions = predictions[valid_mask]
valid_mask = valid_mask.flatten()
perf_y_col_mask = perf_y_col[valid_mask]


true_values = perf_y_col_mask[:,0].astype(np.int32)
pred_values = valid_predictions.flatten()
pred_values_int = np.rint(valid_predictions).flatten().astype(np.int32)

model_win_prob = (1.0*(true_values == pred_values_int).sum()) / (true_values.shape[0])
print(model_win_prob)

# Using XGBoost

In [ ]:
conts_train.shape

In [13]:
# Fit the model
param_grid = {
    'learning_rate': [ 0.008, 0.01, 0.03],           # Removed 0.5, 1 as they're often too aggressive
    'max_depth': [3, 6, 9],                      # Simplified to 3 values, covering shallow to deep
    'n_estimators': [100, 200],                  # Removed extremes, these are most common sweet spots
    'subsample': [0.8, 1.0],                     # Removed 0.6 as it might be too aggressive for this dataset size
    'colsample_bytree': [0.8, 1.0],              # Same as above
    'min_child_weight': [1, 3],                  # Removed 5 as it might be too restrictive
    # 'monotone_constraints': [monotone_constraints_tuple, no_monotone_constraints_tuple]
    # 'monotone_constraints': [no_monotone_constraints_tuple]
}
aparam_grid = {
    'learning_rate': [0.005, 0.01, 0.05],        # Removed 0.5, 1 as they're often too aggressive
    'max_depth': [3],                         # Simplified to 3 values, covering shallow to deep
    'n_estimators': [300, 350, 400],             # Removed extremes, these are most common sweet spots
    'subsample': [0.5, 0.6, 0.7],                # Removed 0.6 as it might be too aggressive for this dataset size
    'colsample_bytree': [0.5, 0.6, 0.7],         # Same as above
    'min_child_weight': [3, 4],                  # Removed 5 as it might be too restrictive
    # 'monotone_constraints': [monotone_constraints_tuple, no_monotone_constraints_tuple]
}

# model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model = XGBRegressor(objective='reg:squarederror')

# Create a custom scorer using the F1 score
# f1_scorer = make_scorer(f1_score, average='weighted')
# Tune hyperparameters using GridSearchCV with the custom F1 scorer
# grid_search = GridSearchCV(model, param_grid, scoring=f1_scorer, cv=5, verbose=1)
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)
grid_search.fit(conts_train, y_train)


Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
        

In [14]:
print(grid_search.best_params_)
# No monotone constraints
# 60.15%, {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}
# 65% {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 0.8}

# Monotone constraints
# {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'monotone_constraints': (1, 1, 1, 1, 1, 1, 0, 1, 1, -1, -1, 0, 1, 1, -1, -1, -1, -1, 0, 1, -1, 1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1), 'n_estimators': 200, 'subsample': 1.0}

{'colsample_bytree': 1.0, 'learning_rate': 0.03, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}


In [12]:

# train final model w/ early stopping
model = XGBClassifier(
    eval_metric='logloss',
    early_stopping_rounds=10,
    # **grid_search.best_params_
    # **{'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 0.8}
    # **{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'monotone_constraints': (1, 1, 1, 1, 1, 1, 0, 1, 1, -1, -1, 0, 1, 1, -1, -1, -1, -1, 0, 1, -1, 1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1), 'n_estimators': 90, 'subsample': 1.0},
    
    # **{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'monotone_constraints': (1, 1, 1, 1, 1, 1, 0, 1, 1, -1, -1, 0, 1, 1, -1, -1, -1, -1, 0, 1, -1, 1, -1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1), 'n_estimators': 200, 'subsample': 1.0}

    # **grid_search.best_params_,
    # **{'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 3, 'monotone_constraints': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'n_estimators': 200, 'subsample': 0.8}
    # 66.5%, dd 31.4%
    # {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 3, 'monotone_constraints': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'n_estimators': 200, 'subsample': 0.8}
    # 66.5%, dd 30.9%
    # {'colsample_bytree': 1.0, 'learning_rate': 0.005, 'max_depth': 5, 'min_child_weight': 3, 'monotone_constraints': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'n_estimators': 300, 'subsample': 0.6}
)
model = XGBRegressor(
    objective='reg:squarederror', 
    # **grid_search.best_params_,
    # 67.3 w/ kelly adjustments 0.2, 0.01
    # {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 4, 'monotone_constraints': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 'n_estimators': 350, 'subsample': 0.6}
    # 67.2, dd 28.68 kelly 0.25, 0.014
    # **{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 350, 'subsample': 0.5}
    
   # **{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 300, 'subsample': 0.5}
    **{'colsample_bytree': 1.0, 'learning_rate': 0.03, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}


)
model.fit(
    conts_train,
    y_train,
    eval_set=[(conts_train, y_train)], # , (holdout_conts, holdout_y)
    verbose=True
)

[0]	validation_0-rmse:0.49255
[1]	validation_0-rmse:0.49108
[2]	validation_0-rmse:0.48967
[3]	validation_0-rmse:0.48835
[4]	validation_0-rmse:0.48711
[5]	validation_0-rmse:0.48584
[6]	validation_0-rmse:0.48473
[7]	validation_0-rmse:0.48363
[8]	validation_0-rmse:0.48256
[9]	validation_0-rmse:0.48160
[10]	validation_0-rmse:0.48067
[11]	validation_0-rmse:0.47977
[12]	validation_0-rmse:0.47888
[13]	validation_0-rmse:0.47808
[14]	validation_0-rmse:0.47732
[15]	validation_0-rmse:0.47657
[16]	validation_0-rmse:0.47587
[17]	validation_0-rmse:0.47518
[18]	validation_0-rmse:0.47454
[19]	validation_0-rmse:0.47394
[20]	validation_0-rmse:0.47332
[21]	validation_0-rmse:0.47277
[22]	validation_0-rmse:0.47221
[23]	validation_0-rmse:0.47169
[24]	validation_0-rmse:0.47117
[25]	validation_0-rmse:0.47069
[26]	validation_0-rmse:0.47016
[27]	validation_0-rmse:0.46968
[28]	validation_0-rmse:0.46928
[29]	validation_0-rmse:0.46889
[30]	validation_0-rmse:0.46850
[31]	validation_0-rmse:0.46809
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
# perf_conts.shape
perf_y_col.shape
perf_y_col[0]

array([0.        , 8.57      , 1.08      , 3.98666667, 1.23444444])

In [16]:
# Make predictions for test data
# model = grid_search.best_estimator_
y_pred = model.predict(perf_conts)

nfl_utils.backtest_model(model, perf_conts, perf_y_col, perf_date_col, initial_capital=1000, position_size=0.1, 
                   confidence_threshold=0.0, show_plot=True, max_won_odds=12.2)

ValueError: Feature shape mismatch, expected: 59, got 48

In [ ]:
# Make confusion matrix
cm = confusion_matrix(perf_y_col[:,0], y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## Save XGBoost model

In [ ]:
model.save_model('xgboost_model.bin')